In [24]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import expon
from astropy.table import Table

In [25]:
def IDmatch(ID1, ID2):

    ID1=np.array(ID1)
    ID2=np.array(ID2)
# compare two ID list ID1 and ID2
# idx_id[i] points to the index of particle in the second list or is
# equal to -1 when id on the ID2 list was not found
    idx_id= np.ndarray(shape=(len(ID1)),dtype=int) #lonarr(n_elements(ID1))
    idx_id[:] = -1
    ind1 = np.argsort(ID1)
    ind2 = np.argsort(ID2)
    s2 = 0
    s2N = len(ID2)
    break_loop=0

    for s1 in range(0, len(ind1)):
        while(ID2[ind2[s2]]<ID1[ind1[s1]]):
            s2=s2+1
            if (s2>=s2N):
                break_loop=1
                break       
        if (break_loop==1):
            break
        if (ID2[ind2[s2]]==ID1[ind1[s1]]): 
            idx_id[ind1[s1]]=ind2[s2]

    ind1 = 0
    ind2 = 0

    idx = np.where(idx_id>-1)
    idx=idx[0]

    return idx

In [26]:
def trackingr(glist, rmin, rmax, snap,name):
    start = glist.index(snap)
    Disk1 = Table.read("/mnt/d/DiskActions2/DiskActions_"+glist[start]+".fits", format = 'fits')
    Jrdisk_1 = Disk1["Jr"]
    Jphidisk_1 = Disk1["Jphi"]
    Jzdisk_1 = Disk1["Jz"]
    
    Diskphspace_1 = Table.read("/mnt/d/PhaseSpaceDisk2/PhaseSpaceDisk_"+glist[start]+".fits", format = 'fits')
    Xdisk_1 = Diskphspace_1["X"]
    Ydisk_1 = Diskphspace_1["Y"]
    Zdisk_1 = Diskphspace_1["Z"]
    Vxdisk_1 = Diskphspace_1["U"]
    Vydisk_1 = Diskphspace_1["V"]
    Vzdisk_1 = Diskphspace_1["W"]
    id_disk_1 = Diskphspace_1["ID"]
    
    R = np.sqrt(Xdisk_1**2 + Ydisk_1**2)
    
    ichoose = np.where((R<rmax) & (R>rmin))

    newX_8 = Xdisk_1[ichoose]
    newY_8 = Ydisk_1[ichoose]
    newZ_8 = Zdisk_1[ichoose]
    newVx_8 = Vxdisk_1[ichoose]
    newVy_8 = Vydisk_1[ichoose]
    newVz_8 = Vzdisk_1[ichoose]

    newJr_8 = Jrdisk_1[ichoose]
    newJphi_8 = Jphidisk_1[ichoose]
    newJz_8 = Jzdisk_1[ichoose]

    ids_8 = id_disk_1[ichoose]
    
    Xlist = np.zeros((len(glist),len(newX_8)))
    Ylist = np.zeros((len(glist),len(newY_8)))
    Zlist = np.zeros((len(glist),len(newZ_8)))
    Vxlist = np.zeros((len(glist),len(newVx_8)))
    Vylist = np.zeros((len(glist),len(newVy_8)))
    Vzlist = np.zeros((len(glist),len(newVz_8)))
    Jrlist = np.zeros((len(glist),len(newJr_8)))
    Jphilist = np.zeros((len(glist),len(newJphi_8)))
    Jzlist = np.zeros((len(glist),len(newJz_8)))
    
    Xlist[0,:] = newX_8
    Ylist[0,:] = newY_8
    Zlist[0,:] = newZ_8
    Vxlist[0,:] = newVx_8
    Vylist[0,:] = newVy_8
    Vylist[0,:] = newVz_8
    Jrlist[0,:] = newJr_8
    Jphilist[0,:] = newJphi_8
    Jzlist[0,:] = newJz_8
    
    count = range(len(glist) - 1)
    l = len(newX_8)
    for gal, c in zip(glist[1:], count):
        Disk = Table.read("/mnt/d/DiskActions2/DiskActions_"+gal+".fits", format = 'fits')
        Jrdisk = Disk["Jr"]
        Jphidisk = Disk["Jphi"]
        Jzdisk = Disk["Jz"] 
        
        Diskphspace = Table.read("/mnt/d/PhaseSpaceDisk2/PhaseSpaceDisk_"+gal+".fits", format = 'fits')
        Xdisk = Diskphspace["X"]
        Ydisk = Diskphspace["Y"]
        Zdisk = Diskphspace["Z"]
        Vxdisk = Diskphspace["U"]
        Vydisk = Diskphspace["V"]
        Vzdisk = Diskphspace["W"]
        id_disk = Diskphspace["ID"]
        
        ichoose2 = IDmatch(id_disk, ids_8)

        newX2_8 = Xdisk[ichoose2]
        newY2_8 = Ydisk[ichoose2]
        newZ2_8 = Zdisk[ichoose2]
        newVx2_8 = Vxdisk[ichoose2]
        newVy2_8 = Vydisk[ichoose2]
        newVz2_8 = Vzdisk[ichoose2]
        newJr2_8 = Jrdisk[ichoose2]
        newJphi2_8 = Jphidisk[ichoose2]
        newJz2_8 = Jzdisk[ichoose2]
        
        #For the later snaps the calculation of actions failed for some stars, so there is a chance
        #the star we are tracking is not in the file. This algorithm assumes that whenever that
        #happens, it is the last star of the array (which is why i remove the last columns)
        #This assumption may not be right, in which case I need to figure out a way to make sure 
        #columns in the list arrays are all the same star ID
        if l > len(newX2_8):
            ntr = l - len(newX2_8)
            ctk = range(l-ntr)
            Xlist = Xlist[:,ctk]
            Ylist = Ylist[:,ctk]
            Zlist = Zlist[:,ctk]
            Vxlist = Vxlist[:,ctk]
            Vylist = Vylist[:,ctk]
            Vzlist = Vzlist[:,ctk]
            Jrlist = Jrlist[:,ctk]
            Jphilist = Jphilist[:,ctk]
            Jzlist = Jzlist[:,ctk]
            l = l - ntr
        
        if len(newX2_8) > l:
            ntr = len(newX2_8) - l
            ctk = range(len(newX2_8)-ntr)
            newX2_8 = newX2_8[ctk]
            newY2_8 = newY2_8[ctk]
            newZ2_8 = newZ2_8[ctk]
            newVx2_8 = newVx2_8[ctk]
            newVy2_8 = newVy2_8[ctk]
            newVz2_8 = newVz2_8[ctk]
            newJr2_8 = newJr2_8[ctk]
            newJphi2_8 = newJphi2_8[ctk]
            newJz2_8 = newJz2_8[ctk]
            
        Xlist[c+1,:] = newX2_8
        Ylist[c+1,:] = newY2_8
        Zlist[c+1,:] = newZ2_8
        Vxlist[c+1,:] = newVx2_8
        Vylist[c+1,:] = newVy2_8
        Vzlist[c+1,:] = newVz2_8
        Jrlist[c+1,:] = newJr2_8
        Jphilist[c+1,:] = newJphi2_8
        Jzlist[c+1,:] = newJz2_8
    
    np.save('/mnt/d/tracking/X_'+name+'.npy', Xlist)
    np.save('/mnt/d/tracking/Y_'+name+'.npy', Ylist)
    np.save('/mnt/d/tracking/Z_'+name+'.npy', Zlist)
    np.save('/mnt/d/tracking/Jr_'+name+'.npy', Jrlist)
    np.save('/mnt/d/tracking/Jphi_'+name+'.npy', Jphilist)
    np.save('/mnt/d/tracking/Jz_'+name+'.npy', Jzlist)
    np.save('/mnt/d/tracking/Vx_'+name+'.npy', Vxlist)
    np.save('/mnt/d/tracking/Vy_'+name+'.npy', Vylist)
    np.save('/mnt/d/tracking/Vz_'+name+'.npy', Vzlist)

In [27]:
def trackingbox(glist, xmin, xmax, ymin, ymax, snap,name):
    start = glist.index(snap)
    Disk1 = Table.read("/mnt/d/DiskActions2/DiskActions_"+glist[start]+".fits", format = 'fits')
    Jrdisk_1 = Disk1["Jr"]
    Jphidisk_1 = Disk1["Jphi"]
    Jzdisk_1 = Disk1["Jz"]
    
    Diskphspace_1 = Table.read("/mnt/d/PhaseSpaceDisk2/PhaseSpaceDisk_"+glist[start]+".fits", format = 'fits')
    Xdisk_1 = Diskphspace_1["X"]
    Ydisk_1 = Diskphspace_1["Y"]
    Zdisk_1 = Diskphspace_1["Z"]
    Vxdisk_1 = Diskphspace_1["U"]
    Vydisk_1 = Diskphspace_1["V"]
    Vzdisk_1 = Diskphspace_1["W"]
    id_disk_1 = Diskphspace_1["ID"]
    
    ichoose = np.where((Xdisk_1<xmax) & (Xdisk_1>xmin) & (Ydisk_1<ymax) & (Ydisk_1>ymin))
    #mu, std = norm.fit(jphi[ichoose])
    
    #vmin = mu - 0.03*std
    #vmax = mu + 0.03*std
    
    #ichoose = np.where((Jphidisk_1<vmax) & (Jphidisk_1>vmin))
    newX_8 = Xdisk_1[ichoose]
    newY_8 = Ydisk_1[ichoose]
    newZ_8 = Zdisk_1[ichoose]
    newVx_8 = Vxdisk_1[ichoose]
    newVy_8 = Vydisk_1[ichoose]
    newVz_8 = Vzdisk_1[ichoose]

    newJr_8 = Jrdisk_1[ichoose]
    newJphi_8 = Jphidisk_1[ichoose]
    newJz_8 = Jzdisk_1[ichoose]

    ids_8 = id_disk_1[ichoose]
    
    Xlist = np.zeros((len(glist),len(newX_8)))
    Ylist = np.zeros((len(glist),len(newY_8)))
    Zlist = np.zeros((len(glist),len(newZ_8)))
    Vxlist = np.zeros((len(glist),len(newVx_8)))
    Vylist = np.zeros((len(glist),len(newVy_8)))
    Vzlist = np.zeros((len(glist),len(newVz_8)))
    Jrlist = np.zeros((len(glist),len(newJr_8)))
    Jphilist = np.zeros((len(glist),len(newJphi_8)))
    Jzlist = np.zeros((len(glist),len(newJz_8)))
    
    count = range(len(glist))
    l = len(ids_8)
    for gal, c in zip(glist, count):
        Disk = Table.read("/mnt/d/DiskActions2/DiskActions_"+gal+".fits", format = 'fits')
        Jrdisk = Disk["Jr"]
        Jphidisk = Disk["Jphi"]
        Jzdisk = Disk["Jz"] 
        
        Diskphspace = Table.read("/mnt/d/PhaseSpaceDisk2/PhaseSpaceDisk_"+gal+".fits", format = 'fits')
        Xdisk = Diskphspace["X"]
        Ydisk = Diskphspace["Y"]
        Zdisk = Diskphspace["Z"]
        Vxdisk = Diskphspace["U"]
        Vydisk = Diskphspace["V"]
        Vzdisk = Diskphspace["W"]
        id_disk = Diskphspace["ID"]
        
        ichoose2 = IDmatch(id_disk, ids_8)

        newX2_8 = Xdisk[ichoose2]
        newY2_8 = Ydisk[ichoose2]
        newZ2_8 = Zdisk[ichoose2]
        newVx2_8 = Vxdisk[ichoose2]
        newVy2_8 = Vydisk[ichoose2]
        newVz2_8 = Vzdisk[ichoose2]
        newJr2_8 = Jrdisk[ichoose2]
        newJphi2_8 = Jphidisk[ichoose2]
        newJz2_8 = Jzdisk[ichoose2]
        
        if l > len(newX2_8):
            ntr = l - len(newX2_8)
            ctk = range(l-ntr)
            Xlist = Xlist[:,ctk]
            Ylist = Ylist[:,ctk]
            Zlist = Zlist[:,ctk]
            Vxlist = Vxlist[:,ctk]
            Vylist = Vylist[:,ctk]
            Vzlist = Vzlist[:,ctk]
            Jrlist = Jrlist[:,ctk]
            Jphilist = Jphilist[:,ctk]
            Jzlist = Jzlist[:,ctk]
            l = l - ntr
        
        if len(newX2_8) > l:
            ntr = len(newX2_8) - l
            ctk = range(len(newX2_8)-ntr)
            newX2_8 = newX2_8[ctk]
            newY2_8 = newY2_8[ctk]
            newZ2_8 = newZ2_8[ctk]
            newVx2_8 = newVx2_8[ctk]
            newVy2_8 = newVy2_8[ctk]
            newVz2_8 = newVz2_8[ctk]
            newJr2_8 = newJr2_8[ctk]
            newJphi2_8 = newJphi2_8[ctk]
            newJz2_8 = newJz2_8[ctk]
            
        Xlist[c,:] = newX2_8
        Ylist[c,:] = newY2_8
        Zlist[c,:] = newZ2_8
        Vxlist[c,:] = newVx2_8
        Vylist[c,:] = newVy2_8
        Vzlist[c,:] = newVz2_8
        Jrlist[c,:] = newJr2_8
        Jphilist[c,:] = newJphi2_8
        Jzlist[c,:] = newJz2_8
        
    np.save('/mnt/d/tracking/X_'+name+'.npy', Xlist)
    np.save('/mnt/d/tracking/Y_'+name+'.npy', Ylist)
    np.save('/mnt/d/tracking/Z_'+name+'.npy', Zlist)
    np.save('/mnt/d/tracking/Jr_'+name+'.npy', Jrlist)
    np.save('/mnt/d/tracking/Jphi_'+name+'.npy', Jphilist)
    np.save('/mnt/d/tracking/Jz_'+name+'.npy', Jzlist)
    np.save('/mnt/d/tracking/Vx_'+name+'.npy', Vxlist)
    np.save('/mnt/d/tracking/Vy_'+name+'.npy', Vylist)
    np.save('/mnt/d/tracking/Vz_'+name+'.npy', Vzlist)

In [28]:
def trackingj(glist, rmin, rmax, snap, name):
    start = glist.index(snap)
    Disk1 = Table.read("/mnt/d/DiskActions2/DiskActions_"+glist[start]+".fits", format = 'fits')
    Jrdisk_1 = Disk1["Jr"]
    Jphidisk_1 = Disk1["Jphi"]
    Jzdisk_1 = Disk1["Jz"]
    
    Diskphspace_1 = Table.read("/mnt/d/PhaseSpaceDisk2/PhaseSpaceDisk_"+glist[start]+".fits", format = 'fits')
    Xdisk_1 = Diskphspace_1["X"]
    Ydisk_1 = Diskphspace_1["Y"]
    Zdisk_1 = Diskphspace_1["Z"]
    Vxdisk_1 = Diskphspace_1["U"]
    Vydisk_1 = Diskphspace_1["V"]
    Vzdisk_1 = Diskphspace_1["W"]
    id_disk_1 = Diskphspace_1["ID"]
    
    R = np.sqrt(Xdisk_1**2 + Ydisk_1**2)
    
    ichoose = np.where((R<rmax) & (R>rmin))
    Jrange = Jphidisk_1[ichoose]
    mean, std = np.nanmean(Jrange), np.nanstd(Jrange)
    vmax = mean + 0.3*std
    vmin = mean - 0.3*std
    
    ichoose = np.where((Jphidisk_1<vmax) & (Jphidisk_1>vmin))

    newX_8 = Xdisk_1[ichoose]
    newY_8 = Ydisk_1[ichoose]
    newZ_8 = Zdisk_1[ichoose]
    newVx_8 = Vxdisk_1[ichoose]
    newVy_8 = Vydisk_1[ichoose]
    newVz_8 = Vzdisk_1[ichoose]

    newJr_8 = Jrdisk_1[ichoose]
    newJphi_8 = Jphidisk_1[ichoose]
    newJz_8 = Jzdisk_1[ichoose]

    ids_8 = id_disk_1[ichoose]
    
    Xlist = np.zeros((len(glist),len(newX_8)))
    Ylist = np.zeros((len(glist),len(newY_8)))
    Zlist = np.zeros((len(glist),len(newZ_8)))
    Vxlist = np.zeros((len(glist),len(newVx_8)))
    Vylist = np.zeros((len(glist),len(newVy_8)))
    Vzlist = np.zeros((len(glist),len(newVz_8)))
    Jrlist = np.zeros((len(glist),len(newJr_8)))
    Jphilist = np.zeros((len(glist),len(newJphi_8)))
    Jzlist = np.zeros((len(glist),len(newJz_8)))
    
    Xlist[0,:] = newX_8
    Ylist[0,:] = newY_8
    Zlist[0,:] = newZ_8
    Vxlist[0,:] = newVx_8
    Vylist[0,:] = newVy_8
    Vylist[0,:] = newVz_8
    Jrlist[0,:] = newJr_8
    Jphilist[0,:] = newJphi_8
    Jzlist[0,:] = newJz_8
    
    count = range(len(glist) - 1)
    l = len(newX_8)
    for gal, c in zip(glist[1:], count):
        Disk = Table.read("/mnt/d/DiskActions2/DiskActions_"+gal+".fits", format = 'fits')
        Jrdisk = Disk["Jr"]
        Jphidisk = Disk["Jphi"]
        Jzdisk = Disk["Jz"] 
        
        Diskphspace = Table.read("/mnt/d/PhaseSpaceDisk2/PhaseSpaceDisk_"+gal+".fits", format = 'fits')
        Xdisk = Diskphspace["X"]
        Ydisk = Diskphspace["Y"]
        Zdisk = Diskphspace["Z"]
        Vxdisk = Diskphspace["U"]
        Vydisk = Diskphspace["V"]
        Vzdisk = Diskphspace["W"]
        id_disk = Diskphspace["ID"]
        
        ichoose2 = IDmatch(id_disk, ids_8)

        newX2_8 = Xdisk[ichoose2]
        newY2_8 = Ydisk[ichoose2]
        newZ2_8 = Zdisk[ichoose2]
        newVx2_8 = Vxdisk[ichoose2]
        newVy2_8 = Vydisk[ichoose2]
        newVz2_8 = Vzdisk[ichoose2]
        newJr2_8 = Jrdisk[ichoose2]
        newJphi2_8 = Jphidisk[ichoose2]
        newJz2_8 = Jzdisk[ichoose2]
        
        #For the later snaps the calculation of actions failed for some stars, so there is a chance
        #the star we are tracking is not in the file. This algorithm assumes that whenever that
        #happens, it is the last star of the array (which is why i remove the last columns)
        #This assumption may not be right, in which case I need to figure out a way to make sure 
        #columns in the list arrays are all the same star ID
        if l > len(newX2_8):
            ntr = l - len(newX2_8)
            ctk = range(l-ntr)
            Xlist = Xlist[:,ctk]
            Ylist = Ylist[:,ctk]
            Zlist = Zlist[:,ctk]
            Vxlist = Vxlist[:,ctk]
            Vylist = Vylist[:,ctk]
            Vzlist = Vzlist[:,ctk]
            Jrlist = Jrlist[:,ctk]
            Jphilist = Jphilist[:,ctk]
            Jzlist = Jzlist[:,ctk]
            l = l - ntr
        
        if len(newX2_8) > l:
            ntr = len(newX2_8) - l
            ctk = range(len(newX2_8)-ntr)
            newX2_8 = newX2_8[ctk]
            newY2_8 = newY2_8[ctk]
            newZ2_8 = newZ2_8[ctk]
            newVx2_8 = newVx2_8[ctk]
            newVy2_8 = newVy2_8[ctk]
            newVz2_8 = newVz2_8[ctk]
            newJr2_8 = newJr2_8[ctk]
            newJphi2_8 = newJphi2_8[ctk]
            newJz2_8 = newJz2_8[ctk]
            
        Xlist[c+1,:] = newX2_8
        Ylist[c+1,:] = newY2_8
        Zlist[c+1,:] = newZ2_8
        Vxlist[c+1,:] = newVx2_8
        Vylist[c+1,:] = newVy2_8
        Vzlist[c+1,:] = newVz2_8
        Jrlist[c+1,:] = newJr2_8
        Jphilist[c+1,:] = newJphi2_8
        Jzlist[c+1,:] = newJz2_8
        
    np.save('/mnt/d/tracking/X_'+name+'.npy', Xlist)
    np.save('/mnt/d/tracking/Y_'+name+'.npy', Ylist)
    np.save('/mnt/d/tracking/Z_'+name+'.npy', Zlist)
    np.save('/mnt/d/tracking/Jr_'+name+'.npy', Jrlist)
    np.save('/mnt/d/tracking/Jphi_'+name+'.npy', Jphilist)
    np.save('/mnt/d/tracking/Jz_'+name+'.npy', Jzlist)
    np.save('/mnt/d/tracking/Vx_'+name+'.npy', Vxlist)
    np.save('/mnt/d/tracking/Vy_'+name+'.npy', Vylist)
    np.save('/mnt/d/tracking/Vz_'+name+'.npy', Vzlist)

In [29]:
#This will take a good while
glist1 = ['010','020','030','040','050','060','070','080','090','100','110','120','130','140','150','160','170','180','190']
glist2 = ['600','605','606','607','608','609','610','611','612','613','614','615','620','625','630','635','640','645','650','655','660','665','670','675','680','685','690','695']

#radius values for impact @ 610
vmin = np.sqrt(1+81)
vmax = np.sqrt(25 +13**2)

#initial snaps
#trackingr(glist1,rmin=3.9,rmax=4.1,snap='010',name='R4Init')
#trackingj(glist1,rmin=3.9,rmax=4.1,snap='010',name='J4Init')
#trackingr(glist1,rmin=7.9,rmax=8.1,snap='010',name='R8Init')
#trackingj(glist1,rmin=7.9,rmax=8.1,snap='010',name='J8Init')
#trackingr(glist1,rmin=11.9,rmax=12.1,snap='010',name='R12Init')
#trackingj(glist1,rmin=11.9,rmax=12.1,snap='010',name='J12Init')

#final snaps
#trackingr(glist2,rmin=3.9,rmax=4.1,snap='600',name='R4Final')
#trackingj(glist2,rmin=3.9,rmax=4.1,snap='600',name='J4Final')
#trackingr(glist2,rmin=7.9,rmax=8.1,snap='600',name='R8Final')
#trackingj(glist2,rmin=7.9,rmax=8.1,snap='600',name='J8Final')
#trackingr(glist2,rmin=11.9,rmax=12.1,snap='600',name='R12Final')
trackingj(glist2,rmin=11.9,rmax=12.1,snap='600',name='J12Final')

#impact
trackingr(glist2,rmin=vmin,rmax=vmax,snap='610',name='RImpRing')
trackingj(glist2,rmin=vmin,rmax=vmax,snap='610',name='JImpRing')
trackingbox(glist2,xmin=9,xmax=13,ymin=-1,ymax=4,snap='610',name='ImpBox')